In [2]:
import pandas as pd

data = pd.read_csv('news_chinese.csv')
data.head(10)

,id,author,source,content,feature,title,url
0,1,王子江 张寒,新华社,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,"{""type"":""体育"",""site"":""新华社"",""url"":""http://home.x...",（体育）题：乒乓女球迷,http://home.xinhua-news.com/gdsd
1,2,夏文辉,新华社,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",哈佛大学为何取消这些新生入选资格？,http://home.xinhua-news.com/gdsd
2,3,张旌,新华社,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",法国议会选举 马克龙有望获“压倒性胜利”,http://home.xinhua-news.com/gdsd
3,4,王衡,新华社,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,"{""type"":""宏观经济"",""site"":""新华社"",""url"":""http://home...",（经济）甘肃集中开工35个重点交通建设项目,http://home.xinhua-news.com/gdsd
4,5,邹峥,新华社,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,"{""type"":""冰球"",""site"":""新华社"",""url"":""http://home.x...",（体育）（2）冰球——国家女子冰球队海外选秀在多伦多展开,http://home.xinhua-news.com/gdsd
5,6,郑焕松,新华社,新华社照片，北京，2017年6月7日\n电影《血战湘江》举行清华大学巡映活动\n6月6日，电...,"{""type"":""社会"",""site"":""新华社"",""url"":""http://home.x...",（文化）（3）电影《血战湘江》举行清华大学巡映活动,http://home.xinhua-news.com/gdsd
6,7,郑焕松,新华社,新华社照片，北京，2017年6月7日\n电影《血战湘江》举行清华大学巡映活动\n6月6日，清...,"{""type"":""社会"",""site"":""新华社"",""url"":""http://home.x...",（文化）（2）电影《血战湘江》举行清华大学巡映活动,http://home.xinhua-news.com/gdsd
7,8,郑焕松,新华社,新华社照片，北京，2017年6月7日\n电影《血战湘江》举行清华大学巡映活动\n6月6日，电...,"{""type"":""社会"",""site"":""新华社"",""url"":""http://home.x...",（文化）（1）电影《血战湘江》举行清华大学巡映活动,http://home.xinhua-news.com/gdsd
8,9,LISELOTTE SABROE,新华社,新华社照片，外代，2017年6月7日\n（外代二线）足球——国际友谊赛：德国平丹麦\n6月6...,"{""type"":""足球"",""site"":""新华社"",""url"":""http://home.x...",[9]（外代二线）足球——国际友谊赛：德国平丹麦,http://home.xinhua-news.com/gdsd
9,10,LISELOTTE SABROE,新华社,新华社照片，外代，2017年6月7日\n（外代二线）足球——国际友谊赛：德国平丹麦\n6月6...,"{""type"":""足球"",""site"":""新华社"",""url"":""http://home.x...",[7]（外代二线）足球——国际友谊赛：德国平丹麦,http://home.xinhua-news.com/gdsd


In [3]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


In [1]:
!conda list pyltp

# packages in environment at E:\Soft\Anaconda3:
#
# Name                    Version                   Build  Channel


In [6]:
from pyltp import Segmentor
segmentor = Segmentor()
segmentor.load("/path/to/your/cws/model")
words = segmentor.segment("元芳你怎么看")
print("|".join(words))
segmentor.release()

ModuleNotFoundError: No module named 'pyltp'

In [3]:
import jieba
segs = jieba.cut("元芳你怎么看")
print(segs)


<generator object Tokenizer.cut at 0x00000256B28FFA98>
